In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("C://workspace//project_data//lata//covid.csv")
df.head()

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,04-05-2020,02-05-2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,06-04-2020,01-04-2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10-04-2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2


In [4]:
df.describe()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
count,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000,566602.000000
mean,1.506726,1.215165,76.562952,1.846262,42.622483,50.400692,2.210633,2.280221,2.265029,2.319628,2.145774,2.410030,2.286157,2.138905,2.283765,2.238360,31.573034,1.728651,76.562864
std,0.499955,0.410937,39.058676,0.560939,16.659973,47.501579,5.683523,5.327832,5.334658,5.667381,5.459866,6.489959,5.438405,5.395578,5.393232,5.571901,45.082123,0.658710,39.059060
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,97.000000,2.000000,31.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,97.000000
50%,2.000000,1.000000,97.000000,2.000000,41.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,97.000000
75%,2.000000,1.000000,97.000000,2.000000,53.000000,97.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,99.000000,2.000000,97.000000
max,2.000000,2.000000,99.000000,99.000000,120.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,98.000000,99.000000,3.000000,99.000000


## 외래 환자 제거
## 남성 임신 97을 임신 아님 2로 변경

In [5]:
train_df = df.drop(df.loc[df['patient_type'] == 1].index)

train_df['pregnancy'].replace(97, 2, inplace=True)
train_df['pregnancy'].value_counts()

2     120676
1       1071
98       166
Name: pregnancy, dtype: int64

## 97,98,99 결측치 제거

In [6]:
train_df.iloc[:,6:] = train_df.iloc[:,6:].replace([97,98,99], 3)
#train_df.dropna(inplace=True)
#train_df.iloc[:,6:].value_counts()

"""
#check = []
#for i in train_df.iloc[:,6:]:
#    check.append(train_df[train_df[i]==98])

#print(check)
"""

'\n#check = []\n#for i in train_df.iloc[:,6:]:\n#    check.append(train_df[train_df[i]==98])\n\n#print(check)\n'

## 중복된 id 제거

In [7]:
#temp = []
#train_df['id'] = train_df['id'].astype(str)

# 중복된 id들은 모두 다른사람들이긴 하다. 이것을 지워야 하는지 논의해보자

## 생존/사망 (0, 1) 분류

In [8]:
# 0 생존, 1 사망
train_df['dd_bool'] = np.where(train_df['date_died'] == '9999-99-99', 0, 1)
train_df['dd_bool'].value_counts()

0    89357
1    32556
Name: dd_bool, dtype: int64

## 증상 발현 후 입원에서부터 사망까지 시간

우선은 사망률을 예측 분석하는 것이니까 살아있는 사람들은 고려를 하지 않아 보았다

In [9]:
train_df.entry_date = pd.to_datetime(df.entry_date, dayfirst=True, errors='coerce')
train_df.date_symptoms = pd.to_datetime(df.date_symptoms, dayfirst=True, errors='coerce')
train_df.date_died = pd.to_datetime(df.date_died, dayfirst=True, errors='coerce')
train_df['dura_of_d'] = train_df['date_died'] - train_df['entry_date']

In [11]:
train_df

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu,dd_bool,dura_of_d
2,167386,1,2,2020-04-06,2020-04-01,NaT,2,2,54,2,...,2,2,1,2,2,3,1,2,0,NaT
3,0b5948,2,2,2020-04-17,2020-04-10,NaT,2,1,30,2,...,2,2,2,2,2,3,1,2,0,NaT
4,0d01b5,1,2,2020-04-13,2020-04-13,2020-04-22,2,2,60,2,...,2,1,2,2,2,3,1,2,1,9 days
5,1beec8,2,2,2020-04-16,2020-04-16,2020-04-29,2,1,47,2,...,2,2,2,2,2,3,1,1,1,13 days
6,1.75E+56,2,2,2020-04-22,2020-04-13,NaT,2,2,63,2,...,2,2,2,2,2,3,1,2,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566584,093a47,2,2,2020-05-02,2020-05-02,NaT,2,1,45,2,...,2,2,2,2,2,3,3,2,0,NaT
566590,067bbd,2,2,2020-06-07,2020-05-24,NaT,1,1,68,2,...,2,2,1,2,2,2,3,1,0,NaT
566591,5.60E+40,2,2,2020-05-08,2020-05-08,2020-05-13,1,1,48,2,...,2,2,2,2,2,2,3,1,1,5 days
566593,150615,1,2,2020-05-12,2020-05-09,NaT,2,1,31,3,...,2,2,2,2,2,3,3,2,0,NaT


## 정규화 적용 및 train/test 셋 분리

In [34]:
t_train_raw = train_df[['sex', 'intubed','pneumonia','age','pregnancy', 'diabetes', 'copd', 'asthma','inmsupr','hypertension','other_disease','cardiovascular','obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']].astype(int)
target_raw = train_df['dd_bool']

## 정규화 적용
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(t_train_raw)
x_train_raw = pd.DataFrame(x_scaled)


## train/test 셋 분리
x_train, x_test, target_train, target_test = train_test_split(x_train_raw, target_raw, test_size=0.2)

## 로지스틱 리그레션: 텐서후로우 뉴럴 네트워크 사용

In [36]:
# ''' '''사이의 코드는 텐서후로우 1 버전 환경으로 작성한것입니다
# 배열이 맞지않아 작동하지 않습니다

'''

tf.set_random_seed(777)

x_train = train_df[['sex', 'intubed','pneumonia','age','pregnancy', 'diabetes', 'copd', 'asthma','inmsupr','hypertension','other_disease','cardiovascular','obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'covid_res', 'icu']]
y_train = train_df['dd_bool']

#x = tf.placeholder(tf.float32, shape=[58600, None])
#y = tf.placeholder(tf.float32, shape=[1, None])

#w = tf.Variable(tf.random_normal([1, 58600]), name='weight')
#b = tf.Variable(tf.random_normal([1]), name='bias')

x = tf.constant(dataVar, dtype=tf.float32, shape=[58600, 25])
y = tf.constant()


hypothesis = tf.sigmoid(tf.matmul(x,w) + b)

loss = - tf.reduce_mean(y * tf.log(hypothesis) + (1-y) * tf.log(1-hypothesis))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)

predict = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype=tf.float32))

sess = tf.Session()

'''

## 텐서후로우 2 버전으로 작성된 코드 입니다

dataset = tf.data.Dataset.from_tensor_slices((x_train.values, target_train.values))

for feat, targ in dataset.take(5):
    print('Festures: {}, Target: {}'.format(feat, targ))


Festures: [0.         0.         0.         0.26271186 0.5        0.
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.         1.         0.5       ], Target: 0
Festures: [1.         0.5        0.         0.46610169 0.5        0.
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.5        0.         0.5       ], Target: 0
Festures: [1.         0.5        0.5        0.33898305 0.5        0.5
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        1.         0.         0.5       ], Target: 0
Festures: [1.         0.5        0.         0.57627119 0.5        0.
 0.5        0.5        0.5        0.         0.5        0.5
 0.5        0.5        0.5        1.         0.         0.5       ], Target: 1
Festures: [1.         0.5        0.5        0.20338983 0.5        0.5
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        1. 

In [37]:
"""
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        _, loss_val = sess.run([train, loss], feed_dict={x:x_train, y:y_train})

        if step % 200 == 0:
            print(step, loss_val)

    h,p,a = sess.run([hypothesis, predict, accuracy], feed_dict={x:x_train, y:y_train})

    print("\n예측값:\n", h, "\npredict:\n", p, "\n정확도:",a)
    
"""


train_dataset = dataset.shuffle(len(train_df)).batch(1)

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(58600, activation='sigmoid'),
        tf.keras.layers.Dense(10, activation='sigmoid'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [26]:
model = get_compiled_model()
model.fit(train_dataset, epochs=1)

Train for 121913 steps
121913/121913 [==============================] - 748s 6ms/step - loss: 0.5813 - accuracy: 0.7330


## 로지스틱 모델: sklearn 방법 이용

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## 모델 생성 및 예측

In [58]:
model = LogisticRegression()
model.fit(x_train, target_train)
print(model.score(x_train, target_train))

0.7700092279298677


## 변수의 계수(coefficient) 확인

In [42]:
print(model.coef_)

[[ 0.24860106 -3.27624617 -1.43370242  3.72382829  1.05475518 -0.31495526
   0.21938393  0.50186947 -0.23470519 -0.10634991 -0.33918583  0.18703693
  -0.22940314 -0.56255166  0.12561194  1.41391747 -1.54861278 -0.09574649]]


## 테스트 데이터 예측하기

In [49]:
target_pred = model.predict(x_test)

## 예측된 테스트 데이터 검증

약 77% 예측률


                생존 예측     사망 예측
실제 생존 (0)  

실제 사망 (1)  


In [55]:
accuracy_score(target_test, target_pred)

0.7695935692900792

In [57]:
confusion_matrix(target_test, target_pred)

array([[16695,  1227],
       [ 4391,  2070]], dtype=int64)